In [1]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer

### drop out  

#### 主要目的：
    主要是解决Overfitting的问题
 
#### 主要思路：
    在模型训练时随机让网络某些隐含层节点的权重不工作，不工作的那些节点可以暂时认为不是网络结构的一部分，但是它的权重得保留下来（只是暂时不更新而已），因为下次样本输入时它可能又得工作了。

In [15]:
# 读取数据
dights = load_digits()
X = dights.data
y = dights.target
y = LabelBinarizer().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [8]:
def addLayer(inputs,in_size,out_size,layer_name,activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]))
    Wx_plus_b = tf.matmul(inputs,Weights) + biases
    
    # 这里是新增的dropout,其他和之前没什么差异
    Wx_plus_b = tf.nn.dropout(Wx_plus_b,keep_prob)
    
    if activation_function is None:
        outputs = Wx_plus_b    
    else:
        outputs = activation_function(Wx_plus_b)
        
    return outputs

In [9]:
# 定义dropout的placeholder得
keep_prob = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32,[None,64])
ys = tf.placeholder(tf.float32,[None,10])

In [10]:
# 添加层
lay1 = addLayer(xs,64,50,'lay1',activation_function=tf.nn.tanh)
prediction = addLayer(lay1,50,10,'lay2',activation_function=tf.nn.softmax)

In [12]:
# 确定一个损失函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))

In [13]:
# 训练初始化
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [14]:
# 创建会话
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [16]:
for i in range(500):
    # 记得跑会话的时候传入keep_prob就好~
    sess.run(train_step,feed_dict={xs:X_train,ys:y_train,keep_prob:0.5})